In [ ]:
import csv
import os
import pandas as pd
import sqlite3

Setting variables

In [ ]:
path_wd   = os.getcwd() 
path_root = path_wd.rsplit("\\",1)[0]

path_data = path_root + '\\data\\'
path_etl  = path_root + '\\etl\\'

path_user = os.path.expanduser('~')

In [ ]:
# This function returns a data frame with ["full_path", "item", "path", "file", ("extension")] 
# root_path -> file path in windows format
# extension = [1, 0] -> extension of the file based on the name
def list_files(root_path, extension):
    if extension == 1:
        cols = ["full_path", "item", "path", "file", "extension"]
    else:
        cols = ["full_path", "item", "path", "file"]
    
    df_final = pd.DataFrame(columns = cols)
    df = pd.DataFrame(columns = cols)

    # List with all items into 'root_path'
    list_root_path = os.listdir(root_path)
    
    # Iterates through root_path items list and concat info: [cols] to final df
    for i in range(len(list_root_path)):
        item = list_root_path[i]
        full_path = root_path + item

        if os.path.isdir(full_path):
            path_list = os.listdir(full_path)

            # Iterates through directory items list 
            for j in range(len(path_list)):
                file = path_list[j]
                f_type = file.rsplit(".", 1)[1] # maxsplit parameter = 1 returns 2 values, then select list position value, same logic

                if extension == 1:
                    data = [full_path+'\\'+file, item, full_path, file, f_type]
                else:
                    data = [full_path+'\\'+file, item, full_path, file]

                df.loc[j] = data

        elif os.path.isfile(full_path):
            file = item
            f_type = file.rsplit(".", 1)[1]

            if extension == 1:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-2], root_path, file, f_type]], columns = cols)
            else:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-2], root_path, file]], columns = cols)

        else:
            file = item
            f_type = file.rsplit(".", 1)[1]

            if extension == 1:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-1], root_path, file, f_type]], columns = cols)
            else:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-1], root_path, file]], columns = cols)

        df_final = pd.concat([df_final, df], ignore_index = True)
        df = pd.DataFrame(columns = cols)
    
    return df_final

In [ ]:
# This function returns a data frame object based on listed file
def get_raw_file(full_path):
    if full_path.split('.')[1] == 'csv':
        df_raw = pd.read_csv(full_path, delimiter=",")
    elif full_path.split('.')[1] == 'xlsx':
        df_raw = pd.read_excel(full_path)
    elif full_path.split('.')[1] == 'json':
        df_raw = pd.read_json(full_path)

    return df_raw

In [ ]:
def remove_headers(f_name):
    with open(f_name, 'r') as f:
        lines = f.readlines()

    lines = lines[1:]

    with open(f_name.replace('_formatted.csv', '_to_load.csv'), 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv.reader(lines))

In [ ]:
# List all files in etl directory
df_etl_files_list = list_files(path_etl, 1)
df_list = df_etl_files_list[df_etl_files_list['item'] == 'loading']

In [ ]:
for i in range(len(df_list)):
    f_name = df_list['full_path'].iloc[i]
    df = remove_headers(f_name)